In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F0'
import chipwhisperer as cw
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    target = cw.target(scope)
except IOError:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope)

print("INFO: Found ChipWhisperer😍")

if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None
    
import time
time.sleep(0.05)
scope.default_setup()
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.05)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 30                       
scope.gain.db                            changed from 5.5                       to 24.8359375               
scope.adc.basic_mode                     changed from low                       to rising_edge              
scope.adc.samples                        changed from 24400                     to 5000                     
scope.adc.trig_count                     changed from 5552561                   to 26161582                 
scope.clock.adc_src                      changed from clkgen_x1                 to clkgen_x4                
scope.clock.adc_freq                     changed from 0                         to 29538459                 
scope.clock.adc_rate                     changed from 0.0                       to 29538459.0        

In [42]:
%%bash
pwd
cd mcu/simpleserial-base/
make PLATFORM=CW308_STM32F0 CRYPTO_TARGET=NONE clean
make PLATFORM=CW308_STM32F0 CRYPTO_TARGET=NONE

/home/lizeren/Desktop/chipwhisperer-study
SS_VER set to SS_VER_1_1
SS_VER set to SS_VER_1_1
.
Welcome to another exciting ChipWhisperer target build!!
.
Cleaning project:
rm -f -- simpleserial-base-CW308_CC2538.hex simpleserial-base-CW301_AVR.hex simpleserial-base-CW303.hex simpleserial-base-CW304.hex simpleserial-base-CW308_MEGARF.hex simpleserial-base-CW308_SAM4L.hex simpleserial-base-CW308_STM32F0.hex simpleserial-base-CW308_STM32F1.hex simpleserial-base-CW308_STM32F2.hex simpleserial-base-CW308_STM32F3.hex simpleserial-base-CW308_STM32F4.hex simpleserial-base-CW308_K24F.hex simpleserial-base-CW308_NRF52.hex simpleserial-base-CW308_AURIX.hex simpleserial-base-CW308_SAML11.hex simpleserial-base-CW308_EFM32TG11B.hex simpleserial-base-CWLITEARM.hex simpleserial-base-CWLITEXMEGA.hex simpleserial-base-CWNANO.hex simpleserial-base-CWHUSKY.hex simpleserial-base-CW308_K82F.hex simpleserial-base-CW308_PSOC62.hex simpleserial-base-CW308_IMXRT1062.hex simpleserial-base-CW308_FE310.hex simplese

In [43]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "mcu/simpleserial-base/simpleserial-base-CW308_STM32F0.hex")


Detected unknown STM32F ID: 0x440
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5167 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5167 bytes


In [44]:
msg = bytearray([0]*16) #simpleserial uses bytearrays
target.simpleserial_write('p', msg)
# print(target.simpleserial_read('r', 16))

In [36]:
def capture_trace(_ignored=None):
    ktp = cw.ktp.Basic()
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text).wave

In [15]:
wave = capture_trace()
print("✔️ OK to continue!")

(ChipWhisperer Target WARNING|File SimpleSerial.py:454) Unexpected start to command: z


✔️ OK to continue!


In [16]:
cw.plot(wave)


:Curve   [x]   (y)

In [21]:
wave2 = capture_trace()

cw.plot(wave2)

(ChipWhisperer Target WARNING|File SimpleSerial.py:454) Unexpected start to command: z


:Curve   [x]   (y)

In [22]:
cw.plot(wave) * cw.plot(wave2)


:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)

In [41]:
wave3 = capture_trace()
cw.plot(wave3)

(ChipWhisperer Target WARNING|File SimpleSerial.py:454) Unexpected start to command: z


:Curve   [x]   (y)

In [45]:
wave4 = capture_trace()
cw.plot(wave4) * cw.plot(wave3)

(ChipWhisperer Target WARNING|File SimpleSerial.py:454) Unexpected start to command: z


:Overlay
   .Curve.I  :Curve   [x]   (y)
   .Curve.II :Curve   [x]   (y)